**Reasoning**:
The previous model ID was incorrect. I need to find a publicly available, 4-bit quantized Llama 3 8B model on Hugging Face Hub that is known to work well on Colab and try loading that.



In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit" # A readily available 4-bit quantized model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
    trust_remote_code=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

## Tokenizer loading

### Subtask:
Load the corresponding tokenizer for the selected model.


**Reasoning**:
Load the tokenizer for the chosen model.



In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

## Pipeline setup

### Subtask:
Set up a text generation pipeline using `transformers`.


**Reasoning**:
Import the `pipeline` function and create a text generation pipeline using the loaded model and tokenizer.



In [9]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


## Gradio interface creation

### Subtask:
Create a simple Gradio interface for the chatbot.


**Reasoning**:
Define the Gradio interface components and the chatbot response function using the previously loaded model and tokenizer.



In [10]:
import gradio as gr

def chatbot_response(message, history):
    # Format the input for the model (adjust based on the model's expected format)
    prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
    response = pipe(prompt, max_new_tokens=256)[0]['generated_text']

    # Extract only the assistant's response
    # This is a simple approach; more robust parsing might be needed depending on the model's output
    response_start = response.find("<|start_header_id|>assistant<|end_header_id|>")
    if response_start != -1:
        response = response[response_start + len("<|start_header_id|>assistant<|end_header_id|>") + 1:].strip()

    return response

# Define the Gradio interface
iface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Enter your message here..."),
    outputs="textbox",
    title="Customer Service Chatbot",
    description="Interact with a free and open-source LLM for customer service.",
    examples=[["What is your return policy?"], ["How can I track my order?"]]
)

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected 2 arguments for function <function chatbot_response at 0x7cf26809fb00>, received 1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1032: UserWarning: Expected at least 2 arguments for function <function chatbot_response at 0x7cf26809fb00>, received 1.
  warnings.warn(


**Reasoning**:
The previous code block generated a warning because the `chatbot_response` function is expected to take two arguments (`message` and `history`) when used with Gradio's chatbot component, but the interface was set up for a single input textbox. The subtask is to create a simple Gradio interface, which should include handling chat history. I will modify the function signature and the Gradio interface to use the `gr.ChatInterface` which is designed for chatbot applications and automatically handles history.



In [11]:
import gradio as gr

def chatbot_response(message, history):
    # The history is a list of tuples, each tuple is (user_message, bot_response)
    # We need to format the entire conversation history for the model
    conversation = "".join([f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n{bot}<|eot_id|>" for user, bot in history])
    conversation += f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"

    response = pipe(conversation, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)[0]['generated_text']

    # Extract only the assistant's response from the potentially longer generated text
    response_start = response.rfind("<|start_header_id|>assistant<|end_header_id|>")
    if response_start != -1:
        response = response[response_start + len("<|start_header_id|>assistant<|end_header_id|>") + 1:].strip()
        # Remove any trailing end-of-turn or end-of-text tokens if the model generated them
        response = response.split("<|eot_id|>")[0].strip()

    return response

# Define the Gradio ChatInterface
iface = gr.ChatInterface(
    fn=chatbot_response,
    title="Customer Service Chatbot",
    description="Interact with a free and open-source LLM for customer service.",
    examples=[["What is your return policy?"], ["How can I track my order?"]]
)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


## Run the gradio app

### Subtask:
Launch the Gradio app.


**Reasoning**:
Launch the Gradio interface to make the chatbot accessible.



In [12]:
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f99e7cf20f1043336b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Summary:

### Data Analysis Key Findings

*   A free and open-source language model suitable for Google Colab's resources, specifically `unsloth/llama-3-8b-Instruct-bnb-4bit`, was successfully loaded using 4-bit quantization.
*   The corresponding tokenizer for the selected model was loaded.
*   A text generation pipeline was successfully set up using the loaded model and tokenizer, configured to utilize the available GPU.
*   A Gradio `ChatInterface` was created, including a function to handle conversation history and generate responses using the text generation pipeline.
*   The Gradio application was launched, providing a public URL for interaction with the customer service chatbot.

### Insights or Next Steps

*   The current implementation extracts the assistant's response by searching for a specific header and splitting on end-of-turn tokens. Further refinement of the output parsing logic could improve robustness against unexpected model output formats.
*   Exploring techniques like prompt engineering or fine-tuning on customer service dialogues could enhance the chatbot's performance and relevance for specific customer service scenarios.
